# Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2
import os
import glob
import sys
import time
import numpy as np
from Shapley import ShapNN
from Shapley import CShapNN
from DShap import DShap
import matplotlib.pyplot as plt
import sklearn
from shap_utils import *
from sklearn.utils import shuffle

from tensorflow.keras.preprocessing.image import load_img, img_to_array

%matplotlib inline
MEM_DIR = './'

Instructions for updating:
non-resource variables are not supported in the long term


# Load In Images

In my cwd, I have two files: one containing cat pictures and another containing 
dog pictures. Here I load them into a a list of arrays so that I can create my train/test sets later. I also create a list of labels such that each time I load a cat image into my list of arrays, the corresponding index in the "labels" list will have a 0 (and a 1 for each dog image). I will just load in 25 from each class for now.

In [2]:
logits = []

labels = []

for file in os.listdir('cat_train'):
        
        logits.append(img_to_array(load_img('cat_train\\'+file, target_size = (227,227))))
        
        labels.append(0)
        
        if len(labels) == 25:
            break
 
for file in os.listdir('dog_train'):
    
    logits.append(img_to_array(load_img('dog_train\\'+file, target_size = (227,227))))
    
    labels.append(1)
    
    if len(labels) == 50:
        break

# Create train/test set

Here I create an array of all my image arrays (and labels), and then shuffle the arrays so that they are in random order. Then I create a test set of the last 10 arrays of my train set, and then delete them from my train set.

In [3]:
X = np.array(logits)
y = np.array(labels)

X, y = shuffle(X, y, random_state=0)

X_test = (X[40:50])
y_test = (y[40:50])

X = (np.delete(X, axis=0, obj = slice(40,50)))
y = (np.delete(y, axis= 0, obj = slice(40,50)))

In [4]:
print("Shape of training images:", X.shape, "  Shape of training labels:", y.shape)

print("Shape of testing images:", X_test.shape, "  Shape of testing labels:", y_test.shape)

Shape of training images: (40, 227, 227, 3)   Shape of training labels: (40,)
Shape of testing images: (10, 227, 227, 3)   Shape of testing labels: (10,)


# Create Model Architecture

The model type "conv" loops through the convolution layers first, then goes to the dense layers. Here is what each argument means:

kernel_sizes: List of integers. Each integer represents the side length of a square filter for a convolution layer. For example, if my kernel_sizes is [3,4], then the first convolution layer will have a filter of size (3,3,number of channels of input image). The second convolution layer will have a filter of size (4,4,number of filters in first layer).

channels: List of integers. Each integer represents the number of filters in each layer. For example, if my channels is [10,20] and my kernel_sizes is [3,4], the first convolution layer will have 10 filters of size (3,3,number of channels of input image). The second convolution layer will have 20 filters of size (4,4,10). You do not need to make channels[0] equal to number of channels in your images, as that is already done for you.

hidden_units: List of integers. Each integer represents the number of nodes in each dense layer (which all occur after the final conv layer). The final dense layer is built for you already, so you do not have to worry about making hidden_units[-1] equal to the number of classes you have - it can be whatever you want.

Everything else should work as it does with other models. A couple things to note: There is something inside the library built to handle images with size (number of images, height, width), meaning there is 1 channel. However, I could not get it to work yet. Also, if the directory you are saving stuff to already exists, make sure to set "overwrite" to True. If you don't, it makes dataset for you and can cause some errors (I'm sure it has a purpose, just do not know how it works).

Also, gshap is not built to run for convolutional networks (even though it would be way faster than tmc shap). So if you run with g_run set to True, it will be a fully dense network with architecture of hidden_units. Not sure how it resizes the images for that to work, but it does it somehow.

Also note that this algorithm is very slow. I am running this shallow network on an RTX 2080 and it takes a few hours to converge on values for 50 images. If gshap every gets support for model type 'conv', it will run much faster.



In [5]:
problem, model = 'classification', 'conv'
kernel_sizes = [3,3]
channels = [16,24]
hidden_units = [128,64]
num_test = 10
directory = './temp'

In [6]:
dshap = DShap(X, y, X_test, y_test, num_test, model_family=model, metric='accuracy',
              directory=directory, problem=problem, seed=1, hidden_units = hidden_units, 
              channels = channels, kernel_sizes = kernel_sizes, batch_size = 32, overwrite=True)

In [ ]:
dshap.run(100, 0.1, g_run = False, loo_run = True)

Starting LOO score calculations!
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
LOO values calculated!


In [ ]:
dshap.merge_results()

In [ ]:
convergence_plots(dshap.marginals_tmc)

In [ ]:
dshap.performance_plots([dshap.vals_tmc, dshap.vals_loo], num_plot_markers=20,
                       sources=dshap.sources)